In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import eeweather
import pandas as pd
import datetime
from pytz import timezone
import numpy as np
import re

%matplotlib inline  

In [2]:
#read in the data from the CAISO data pull and save it to df

df = pd.read_csv('C:\\Users\\jlucz\\Documents\\Python Scripts\\master\\CAISO_Data\\Model_Input_Data.csv')
df = pd.DataFrame(df)

In [3]:
#convert TimeStamp to a datetime field which will be needed when we join with other weather data. Currently a string

df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])

In [4]:
df.head()

,INTERVALSTARTTIME_PST,INTERVALSTARTTIME_GMT,INTERVALENDTIME_GMT,OPR_DT,OPR_HR,LMP_Price_Per_MWh,Wind_MW,Solar_MW,Demand_MW,Fuel_Price,TimeStamp,Year,Month,Day,Hour,Time,Weekday,Weekend
0,2019-01-01 00:00:00+00:00,2019-01-01 08:00:00+00:00,2019-01-01T09:00:00-00:00,2019-01-01,1,43.07910,1081.59,0.0,22362.14,4.48,2019-01-01 00:00:00+00:00,2019,1,1,0,00:00:00,1,0
1,2019-01-01 01:00:00+00:00,2019-01-01 09:00:00+00:00,2019-01-01T10:00:00-00:00,2019-01-01,2,41.70186,1073.67,0.0,21443.99,4.48,2019-01-01 01:00:00+00:00,2019,1,1,1,01:00:00,1,0
2,2019-01-01 02:00:00+00:00,2019-01-01 10:00:00+00:00,2019-01-01T11:00:00-00:00,2019-01-01,3,40.45232,1048.92,0.0,20802.71,4.48,2019-01-01 02:00:00+00:00,2019,1,1,2,02:00:00,1,0
3,2019-01-01 03:00:00+00:00,2019-01-01 11:00:00+00:00,2019-01-01T12:00:00-00:00,2019-01-01,4,39.73786,1019.20,0.0,20415.67,4.48,2019-01-01 03:00:00+00:00,2019,1,1,3,03:00:00,1,0
4,2019-01-01 04:00:00+00:00,2019-01-01 12:00:00+00:00,2019-01-01T13:00:00-00:00,2019-01-01,5,40.78106,988.76,0.0,20363.70,4.48,2019-01-01 04:00:00+00:00,2019,1,1,4,04:00:00,1,0


In [11]:
from IPython.display import display

In [38]:
# Not needed, just was very proud of being able to pull values off a pdf using regex!


# #Using the PyPDF2 library, pulled zipcodes off a PDF. We just noticed that the file was outdated. We saved this code in here
# #as we were proud of being able to use a new library and regex effectively.

# #Pull information from PDF
# import PyPDF2

# pdfFile = open('C:\\Users\\jlucz\\Desktop\\Projects\\Nexant Internship\\PGAE_ServiceLocations.pdf', 'rb')
# pdfReader = PyPDF2.PdfFileReader(pdfFile)
# x = pdfReader.getNumPages()
# y = range(x)

# Loops through each page of a pdf and returns a list of all zipcodes found in the pdf. We then add it to a dataframe so all
# zipcodes are in one column.

# data = []
# zips = []

# for i in y:
#     text = pdfReader.getPage(i).extractText()
#     data.append(text)
    
# for i in y:
#     regex = re.compile(r'[0-9]{5}')
#     new = regex.findall(data[i])
#     zips.extend(new)
    
# zipcodes = pd.DataFrame(zips)
# zipcodes.drop_duplicates(keep='first')
# print(zipcodes)


In [39]:
#Checked 2013 data to make sure PG&E increased the number of zip codes and they were relatively similar
#zip2013 = pd.read_csv('C:\\Users\\jlucz\\Desktop\\Projects\\Nexant Internship\\PGE_2013_Q1_ElectricUsageByZip.csv')
#zip2013 = pd.DataFrame(zip2013['ZipCode'])
#zip2013.drop_duplicates(keep='first', inplace=True)

#Build a dataframe to house all zip codes where PG&E operates. Used a list from 2019 to match the year of our CAISO data
zip2019 = pd.read_csv('C:\\Users\\jlucz\\Desktop\\Projects\\Nexant Internship\\PGE_2019_Q4_ElectricUsageByZip.csv')
zip2019 = pd.DataFrame(zip2019['ZIPCODE'])
zip2019.drop_duplicates(keep='first', inplace = True)

zip2019.reset_index(inplace=True)
zip2019.drop('index', axis=1, inplace=True)
zip2019

,ZIPCODE
0,93101
1,93110
2,93117
3,93202
4,93203
...,...
893,95986
894,96009
895,96084
896,96090


In [40]:
#Now we need to convert zipcodes to Latitude and Longitude to be able to use eeweather.rank_stations
#this will allow us to find stations close to each zipcode before running the weather stations through NSRDB

import pgeocode
location = pd.DataFrame()
zip_lat = []
zip_lon = []

for i in zip2019.index:
    nomi = pgeocode.Nominatim('US')
    x = nomi.query_postal_code(str(zip2019.loc[i][0]))
    zip_lat.append(x['latitude'])
    zip_lon.append(x['longitude'])
    
location['latitude'] = zip_lat
location['longitude'] = zip_lon


In [41]:
#Using eeweather.rank_stations we will locate 3 stations that are closest to each zipcode given latitude and longitude of 
#each zipcode PG&E operates. Save these latitudes and longitudes into a dataframe titled weather_coor that will be used
#in our API to return weather data.

station_lat = []
station_lon = []
weather_coor = pd.DataFrame()

for i in location.index:
    x = eeweather.rank_stations(location['latitude'].loc[i], location['longitude'].loc[i])
    for z in x[x['rank'] < 4].index:
        station_lat.append(x['latitude'].loc[z])
        station_lon.append(x['longitude'].loc[z])
    
weather_coor['latitude'] = station_lat
weather_coor['longitude'] = station_lon
weather_coor.drop_duplicates(keep='first', inplace=True)
weather_coor

,latitude,longitude
0,34.400,-119.700
1,34.408,-119.658
2,34.426,-119.843
8,34.467,-120.033
9,36.319,-119.629
...,...,...
2426,39.183,-119.733
2432,39.484,-119.771
2535,34.267,-116.850
2536,34.264,-116.854


In [61]:
#we need to reset the index since we removed an abundance of values when removing duplicates. If we don't, we will run into
#issues when running a for loop and reaching an index value that isn't in the dataframe.

weather_coor.reset_index(drop=True, inplace=True)
weather_coor

,latitude,longitude
0,34.400,-119.700
1,34.408,-119.658
2,34.426,-119.843
3,34.467,-120.033
4,36.319,-119.629
...,...,...
152,39.183,-119.733
153,39.484,-119.771
154,34.267,-116.850
155,34.264,-116.854


In [63]:
#NREL - National Renewable Energy Laboratory
#API that pulls from the NSRDB - National Solar Radiation Database
#https://developer.nrel.gov/docs/solar/nsrdb/python-examples/
# Declare all variables as strings. Spaces must be replaced with '+', i.e., change 'John Smith' to 'John+Smith'.
# Define the lat, long of the location and the year

#This for loop runs every the ranked weather station locations from the created weather_coor dataframe into the API
#to return weather information.
dfall = pd.DataFrame()
for i in weather_coor.index:
    lat, lon, year = weather_coor['latitude'][i],weather_coor['longitude'][i], 2019
# You must request an NSRDB api key from the link above
    api_key = 'IOqZQbrCEqOZZUhxMPjPRmkVqw18TrRa8OUBROcW'
# Set the attributes to extract (e.g., dhi, ghi, etc.), separated by commas.
    attributes = 'ghi,dhi,dni,wind_speed,air_temperature,solar_zenith_angle'
# Choose year of data
    year = '2019'
# Set leap year to true or false. True will return leap day data if present, false will not.
    leap_year = 'false'
# Set time interval in minutes, i.e., '30' is half hour intervals. Valid intervals are 30 & 60.
    interval = '60'
# Specify Coordinated Universal Time (UTC), 'true' will use UTC, 'false' will use the local time zone of the data.
# NOTE: In order to use the NSRDB data in SAM, you must specify UTC as 'false'. SAM requires the data to be in the
# local time zone.
    utc = 'false'
# Your full name, use '+' instead of spaces.
    your_name = 'Jeff+Luczak'
# Your reason for using the NSRDB.
    reason_for_use = 'beta+testing'
# Your affiliation
    your_affiliation = 'ASU'
# Your email address
    your_email = 'jluczak18@gmail.com'
# Please join our mailing list so we can keep you up-to-date on new developments.
    mailing_list = 'true'
# Declare url string
    url = 'https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
# Return just the first 2 lines to get metadata:
    info = pd.read_csv(url, nrows=1)
# See metadata for specified properties, e.g., timezone and elevation
    timezone, elevation = info['Local Time Zone'], info['Elevation']
    dftemp = pd.read_csv('https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes), skiprows=2)
    dftemp['lat'] = lat
    dftemp['lon'] = lon
# Set the time index in the pandas dataframe:
    dftemp = dftemp.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))
    dfall = dfall.append(dftemp)
# take a look
#print('shape:', df.shape)
dfall.head()

,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle,lat,lon
2019-01-01 00:00:00,2019,1,1,0,30,0,0,0,2.7,8.9,167.09,34.4,-119.7
2019-01-01 01:00:00,2019,1,1,1,30,0,0,0,3.0,8.8,157.71,34.4,-119.7
2019-01-01 02:00:00,2019,1,1,2,30,0,0,0,3.2,8.7,145.88,34.4,-119.7
2019-01-01 03:00:00,2019,1,1,3,30,0,0,0,3.2,8.5,133.56,34.4,-119.7
2019-01-01 04:00:00,2019,1,1,4,30,0,0,0,3.1,8.2,121.23,34.4,-119.7


In [66]:
#Now that we have all the data in here, we have more than 1.3 million rows due to 8,760 rows of data for each location
#This groups all the data by day and hour and averages all the weather information

clean_data = dfall.groupby(by=dfall.index).mean()

In [67]:
clean_data

,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle,lat,lon
2019-01-01 00:00:00,2019.0,1.0,1.0,0.0,30.0,0.0,0.0,0.0,2.504459,2.643312,164.502611,37.785166,-121.549191
2019-01-01 01:00:00,2019.0,1.0,1.0,1.0,30.0,0.0,0.0,0.0,2.504459,2.452866,157.212675,37.785166,-121.549191
2019-01-01 02:00:00,2019.0,1.0,1.0,2.0,30.0,0.0,0.0,0.0,2.501911,2.278981,146.510000,37.785166,-121.549191
2019-01-01 03:00:00,2019.0,1.0,1.0,3.0,30.0,0.0,0.0,0.0,2.447771,2.057962,134.873822,37.785166,-121.549191
2019-01-01 04:00:00,2019.0,1.0,1.0,4.0,30.0,0.0,0.0,0.0,2.370064,1.871975,123.048662,37.785166,-121.549191
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,2019.0,12.0,31.0,19.0,30.0,0.0,0.0,0.0,2.933121,4.884713,119.345860,37.785166,-121.549191
2019-12-31 20:00:00,2019.0,12.0,31.0,20.0,30.0,0.0,0.0,0.0,2.767516,4.264331,131.155287,37.785166,-121.549191
2019-12-31 21:00:00,2019.0,12.0,31.0,21.0,30.0,0.0,0.0,0.0,2.665605,3.743949,142.908726,37.785166,-121.549191
2019-12-31 22:00:00,2019.0,12.0,31.0,22.0,30.0,0.0,0.0,0.0,2.597452,3.300000,154.067389,37.785166,-121.549191


In [69]:
#we need to convert the index to include the utc time to match the original dataframe from the CAISO data pull information
clean_data.index = pd.to_datetime(clean_data.index, utc=True)

In [89]:
#Need to reset the index as we can't join on an index value. We rename the index column to StartTime and use that to merge
#with the CAISO data.

#clean_data.reset_index()
clean_data.rename({'index': 'StartTime'}, axis=1, inplace=True)

In [92]:
#merging the two dataframes together
final_df = clean_data.merge(df, left_on = 'StartTime', right_on = 'TimeStamp', how='inner')

In [93]:
final_df

,StartTime,Year_x,Month_x,Day_x,Hour_x,Minute,GHI,DHI,DNI,Wind Speed,...,Demand_MW,Fuel_Price,TimeStamp,Year_y,Month_y,Day_y,Hour_y,Time,Weekday,Weekend
0,2019-01-01 00:00:00+00:00,2019.0,1.0,1.0,0.0,30.0,0.000000,0.000000,0.000000,2.504459,...,22362.14,4.48,2019-01-01 00:00:00+00:00,2019,1,1,0,00:00:00,1,0
1,2019-01-01 01:00:00+00:00,2019.0,1.0,1.0,1.0,30.0,0.000000,0.000000,0.000000,2.504459,...,21443.99,4.48,2019-01-01 01:00:00+00:00,2019,1,1,1,01:00:00,1,0
2,2019-01-01 02:00:00+00:00,2019.0,1.0,1.0,2.0,30.0,0.000000,0.000000,0.000000,2.501911,...,20802.71,4.48,2019-01-01 02:00:00+00:00,2019,1,1,2,02:00:00,1,0
3,2019-01-01 03:00:00+00:00,2019.0,1.0,1.0,3.0,30.0,0.000000,0.000000,0.000000,2.447771,...,20415.67,4.48,2019-01-01 03:00:00+00:00,2019,1,1,3,03:00:00,1,0
4,2019-01-01 04:00:00+00:00,2019.0,1.0,1.0,4.0,30.0,0.000000,0.000000,0.000000,2.370064,...,20363.70,4.48,2019-01-01 04:00:00+00:00,2019,1,1,4,04:00:00,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8647,2019-12-31 10:00:00+00:00,2019.0,12.0,31.0,10.0,30.0,386.738854,88.764331,700.630573,1.658599,...,22102.83,4.26,2019-12-31 10:00:00+00:00,2019,12,31,10,10:00:00,1,0
8648,2019-12-31 11:00:00+00:00,2019.0,12.0,31.0,11.0,30.0,442.904459,108.605096,697.318471,1.522293,...,21371.91,4.26,2019-12-31 11:00:00+00:00,2019,12,31,11,11:00:00,1,0
8649,2019-12-31 12:00:00+00:00,2019.0,12.0,31.0,12.0,30.0,440.057325,121.904459,654.726115,1.550318,...,20924.31,4.26,2019-12-31 12:00:00+00:00,2019,12,31,12,12:00:00,1,0
8650,2019-12-31 13:00:00+00:00,2019.0,12.0,31.0,13.0,30.0,386.980892,117.891720,605.802548,1.589809,...,20909.73,4.26,2019-12-31 13:00:00+00:00,2019,12,31,13,13:00:00,1,0


In [97]:
#Removing duplicate values such as year, month, day, and hour. Also reorganized some of the information so it is cleaner to
#look at
final_df = final_df[['StartTime', 'INTERVALSTARTTIME_PST', 'INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT', 'Year_x', 'Month_x', 'Day_x', 'Hour_x', 'Minute', 
                     'OPR_DT', 'OPR_HR', 'Time', 'Weekday', 'Weekend', 'GHI',
       'DHI', 'DNI', 'Wind Speed', 'Temperature', 'Solar Zenith Angle', 'Wind_MW', 'Solar_MW',


In [98]:
#we are 18 rows short of a full year worth of information due to the change from GMT to PST (9 hour difference)
#This difference effects the first and last day of the year and is why we only have 8,652 rows. 
#(8,670 would be every hour for a full year)   

final_df

,StartTime,INTERVALSTARTTIME_PST,INTERVALSTARTTIME_GMT,INTERVALENDTIME_GMT,Year_x,Month_x,Day_x,Hour_x,Minute,OPR_DT,...,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle,Wind_MW,Solar_MW,Demand_MW,Fuel_Price,LMP_Price_Per_MWh
0,2019-01-01 00:00:00+00:00,2019-01-01 00:00:00+00:00,2019-01-01 08:00:00+00:00,2019-01-01T09:00:00-00:00,2019.0,1.0,1.0,0.0,30.0,2019-01-01,...,0.000000,0.000000,2.504459,2.643312,164.502611,1081.59,0.00,22362.14,4.48,43.07910
1,2019-01-01 01:00:00+00:00,2019-01-01 01:00:00+00:00,2019-01-01 09:00:00+00:00,2019-01-01T10:00:00-00:00,2019.0,1.0,1.0,1.0,30.0,2019-01-01,...,0.000000,0.000000,2.504459,2.452866,157.212675,1073.67,0.00,21443.99,4.48,41.70186
2,2019-01-01 02:00:00+00:00,2019-01-01 02:00:00+00:00,2019-01-01 10:00:00+00:00,2019-01-01T11:00:00-00:00,2019.0,1.0,1.0,2.0,30.0,2019-01-01,...,0.000000,0.000000,2.501911,2.278981,146.510000,1048.92,0.00,20802.71,4.48,40.45232
3,2019-01-01 03:00:00+00:00,2019-01-01 03:00:00+00:00,2019-01-01 11:00:00+00:00,2019-01-01T12:00:00-00:00,2019.0,1.0,1.0,3.0,30.0,2019-01-01,...,0.000000,0.000000,2.447771,2.057962,134.873822,1019.20,0.00,20415.67,4.48,39.73786
4,2019-01-01 04:00:00+00:00,2019-01-01 04:00:00+00:00,2019-01-01 12:00:00+00:00,2019-01-01T13:00:00-00:00,2019.0,1.0,1.0,4.0,30.0,2019-01-01,...,0.000000,0.000000,2.370064,1.871975,123.048662,988.76,0.00,20363.70,4.48,40.78106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8647,2019-12-31 10:00:00+00:00,2019-12-31 10:00:00+00:00,2019-12-31 18:00:00+00:00,2019-12-31T19:00:00-00:00,2019.0,12.0,31.0,10.0,30.0,2019-12-31,...,88.764331,700.630573,1.658599,12.634395,65.140573,45.33,664.95,22102.83,4.26,30.10541
8648,2019-12-31 11:00:00+00:00,2019-12-31 11:00:00+00:00,2019-12-31 19:00:00+00:00,2019-12-31T20:00:00-00:00,2019.0,12.0,31.0,11.0,30.0,2019-12-31,...,108.605096,697.318471,1.522293,13.938217,61.533121,34.49,691.62,21371.91,4.26,28.79112
8649,2019-12-31 12:00:00+00:00,2019-12-31 12:00:00+00:00,2019-12-31 20:00:00+00:00,2019-12-31T21:00:00-00:00,2019.0,12.0,31.0,12.0,30.0,2019-12-31,...,121.904459,654.726115,1.550318,14.677707,61.040955,20.19,711.22,20924.31,4.26,26.26433
8650,2019-12-31 13:00:00+00:00,2019-12-31 13:00:00+00:00,2019-12-31 21:00:00+00:00,2019-12-31T22:00:00-00:00,2019.0,12.0,31.0,13.0,30.0,2019-12-31,...,117.891720,605.802548,1.589809,14.857325,63.737516,10.86,713.13,20909.73,4.26,27.64517
